In [12]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langserve import RemoteRunnable
import json
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents.base import Document
from langchain.tools.retriever import create_retriever_tool
from typing import List, Union
from langchain_community.tools import Tool
import os

In [13]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [14]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

# llm = OllamaFunctions(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", format="json", temperature=0)
eeve = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", temperature=0)
qwen2 = ChatOllama(model="qwen2:latest", temperature=0)

In [15]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# 로컬 DB 불러오기
MY_NEWS_INDEX = "MY_NEWS_INDEX"
vectorstore1 = FAISS.load_local(MY_NEWS_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever1 = vectorstore1.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출
MY_PDF_INDEX = "MY_PDF_INDEX"
vectorstore2 = FAISS.load_local(MY_PDF_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever2 = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출

In [16]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.pydantic_v1 import BaseModel, Field

retriever_tool1 = create_retriever_tool(
    retriever1,
    name="saved_news_search",
    description="엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다",
)

retriever_tool2 = create_retriever_tool(
    retriever2,
    name="pdf_search",
    description="생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다. 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보는 이 도구를 사용해야 한다",
)

tools = [retriever_tool1, retriever_tool2]
tools

[Tool(name='saved_news_search', description='엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x114aef420>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x299ae58d0>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x114aef600>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x299ae58d0>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 Tool(name='pdf_search', description='생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를

In [49]:
prompt_for_select_tool = ChatPromptTemplate.from_messages([
    ("system", """
당신은 인간의 질문에 답변하기 위해 적절한 도구를 선택하는 AI 어시스턴트입니다. 

다음 도구들을 사용할 수 있습니다:
{tools}

인간의 질문을 주의 깊게 분석하고, 가장 적절한 도구를 선택하여 답변하세요. 질문에 따라 여러 도구를 사용해야 할 수도 있습니다.

응답 시 다음 JSON 형식을 엄격히 따라주세요:
```json
[
  {{
    "action": string, // 선택한 도구의 이름 (tool_name)
    "action_input": string // 도구에 입력할 검색어 또는 질문
  }},
  {{
    // 다음 액션 정보
  }}
]
```

응답 지침:
1. 항상 JSON 배열로 응답하세요, 단일 도구를 사용하는 경우에도 마찬가지입니다.
2. 하나의 도구만 필요한 경우, 배열에 하나의 객체만 포함시키세요.
3. 여러 도구가 필요한 경우, 각 도구에 대해 별도의 객체를 배열에 추가하세요.
4. 액션의 순서가 중요한 경우, 배열 내 객체의 순서로 표현하세요.
5. 이 JSON 형식으로만 응답하고, 다른 설명이나 추가 텍스트는 포함하지 마세요.
6. 인간의 질문에 직접 답변하지 말고, 적절한 도구를 선택하여 JSON 형식으로만 응답하세요.

question: {question}

answer: 
"""
    )
])

def get_tools(query):
    """
    사용 가능한 도구들의 이름과 설명을 JSON 형식으로 반환
    """
    # tools 리스트에서 각 도구의 이름, 설명을 딕셔너리 형태로 추출
    tool_info = [{"tool_name": tool.name, "tool_description": tool.description} for tool in tools]
    
    print(f"get_tools / tool_info: {tool_info}")
    
    # tool_info 리스트를 JSON 형식으로 변환하여 반환
    return json.dumps(tool_info, ensure_ascii=False)

chain_for_select_actions = (
    {"tools": get_tools, "question": RunnablePassthrough()}
    | prompt_for_select_tool 
    | qwen2
    | StrOutputParser()
    )

In [50]:
# query = "라마3 성능은?"
# query = "생성형 AI 도입에 따른 규제 연구 책임자는?"
query = "라마3 성능은 어떻게 돼? 그리고 생성형 AI 도입에 따른 규제 연구 책임자는 누구야?"
actions_json = chain_for_select_actions.invoke(query)
actions_json

get_tools / [{'tool_name': 'saved_news_search', 'tool_description': '엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다'}, {'tool_name': 'pdf_search', 'tool_description': '생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다. 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보는 이 도구를 사용해야 한다'}]


'[\n  {\n    "action": "saved_news_search",\n    "action_input": "라마3 성능"\n  },\n  {\n    "action": "pdf_search",\n    "action_input": "생성형 AI 신기술 도입에 따른 선거 규제 연구 책임자"\n  }\n]'

In [55]:
# def get_retriever_by_tool_name(name) -> VectorStoreRetriever:
#     print(f"get_retriever_by_tool_name / name: {name}")
#     for tool in tools:
#         if tool.name == name:
#             # print(tool.func) # functools.partial(<function _get_relevant_documents at 0x1487dd6c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x317e52ea0>, search_kwargs={'k': 5}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')
#             return tool.func.keywords['retriever']
#     return None

# retriever = get_retriever_by_tool_name(selected_tool)
# # retriever.invoke(query)
# retriever

# def get_documents_from_actions(actions_json: str, tools: List[Tool]) -> List[Document]:
def get_documents_from_actions(actions_json, tools):
    """
    Parse the given JSON string of actions, find corresponding retrievers,
    and return documents retrieved by invoking the actions.
    
    :param actions_json: A JSON string containing a list of actions and their inputs
    :param tools: A list of available tools
    :return: A list of documents retrieved from the actions
    """
    # Parse the JSON string
    try:
        actions = json.loads(actions_json)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON string provided")

    # Validate that the parsed object is a list
    if not isinstance(actions, list):
        raise ValueError("The provided JSON should represent a list of actions")

    documents = []

    # Function to get retriever by tool name
    def get_retriever_by_tool_name(name: str) -> VectorStoreRetriever:
        for tool in tools:
            if tool.name == name:
                return tool.func.keywords['retriever']
        return None

    # Process each action
    for action in actions:
        if not isinstance(action, dict) or 'action' not in action or 'action_input' not in action:
            continue  # Skip invalid actions

        tool_name = action['action']
        action_input = action['action_input']
        print(f"get_documents_from_actions / tool_name: {tool_name} / action_input: {action_input}")
        retriever = get_retriever_by_tool_name(tool_name)
        
        if retriever:
            # Invoke the retriever with the action input
            retrieved_docs = retriever.get_relevant_documents(action_input)
            documents.extend(retrieved_docs)

    return documents

get_documents_from_actions(actions_json, tools)

get_documents_from_actions / tool_name: saved_news_search / action_input: 라마3 성능
get_documents_from_actions / tool_name: pdf_search / action_input: 생성형 AI 신기술 도입에 따른 선거 규제 연구 책임자


[Document(page_content="라마 3 벤치마크 결과 (사진=메타)\n\n\n라마 3는 객관식 문제(MMLU)와 코딩(HumanEval)에는 강하지만, 70B의 경우 수학 단어 문제(MATH) 해결이나 대학원생 수준의 객관식 문제(GPQA)에서는 제미나이 프로 1.5에 떨어졌다.\xa0\n특히 인간 선호도에서 경쟁 모델을 앞서는 것으로 알려졌다.\n조언 요청, 브레인스토밍, 분류, 비공개 질문 답변, 코딩, 창의적인 글쓰기, 추출, 공개 질문 답변, 추론, 재작성 및 요약 등 12가지 주요 사용 사례를 포함한 1800개\xa0프롬프트\xa0구축\xa0데이터셋에 대한 인간 평가에서 오픈AI의 'GPT-3.5', 미스트랄 7B, 클로드 3 소네트보다 높게 평가됐다.\n\n\n라마 3 인간 평가 결과 (사진=메타)", metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content='라마 3 인간 평가 결과 (사진=메타)\n\n\n허깅페이스에 따르면, 라마 3는 공개 후 몇시간만에 LLM 리더보드\xa01위에 오르며 역대 가장 빠른 1위 달성 기록을 세웠다.\n또 이전 라마 1과 2를 기반으로 3만개 이상의 새로운 모델이 출시됐으며, 라마 2 모델은 1700억번 다운로드됐다는 통계치도 공개해 눈길을 모았다.\xa0\n다만 라마 3는 완전한 오픈 소스가 아니다.\xa0연구용 및 상업용으로 모두 사용할 수 있지만, 개발자가 다른 생성 모델을 훈련하기 위해 모델을 사용하는 것을 금지한다.\n\n\n메타 AI (사진=메타)', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content='특히 15조개 이상의 토큰을 동원, 학습량이 라마 2 대비 7배 이상 많으며 코드량은 4배 더 많다. 다만 데이터셋은 공

In [56]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
너는 정확하고 신뢰할 수 있는 답변을 제공하는 유능한 업무 보조자야.
아래의 context를 사용해서 question에 대한 답변을 작성해줘.

다음 지침을 따라주세요:
1. 답변은 반드시 한국어로 작성해야 해.
2. context에 있는 정보만을 사용해서 답변해야 해.
3. 정답을 확실히 알 수 없다면 "주어진 정보로는 답변하기 어렵습니다."라고만 말해.
4. 답변 시 추측하거나 개인적인 의견을 추가하지 마.
5. 가능한 간결하고 명확하게 답변해.

# question: 
{question}

# context: 
{context}

# answer:
"""
    ),
])

retrieved_docs = []
def get_page_contents_with_metadata(docs) -> str: 
    global retrieved_docs
    retrieved_docs = docs
    
    result = ""
    for i, doc in enumerate(docs):
        if i > 0:
            result += "\n\n"
        result += f"## 본문: {doc.page_content}\n### 출처: {doc.metadata['source']}"
    return result

def get_retrieved_docs(query) -> str:
    actions_json = chain_for_select_actions.invoke(query)
    docs = get_documents_from_actions(actions_json, tools)
    return get_page_contents_with_metadata(docs)

def get_metadata_sources(docs) -> str: 
    sources = set()
    for doc in docs:
        source = doc.metadata['source']
        is_pdf = source.endswith('.pdf')
        if (is_pdf):
            file_path = doc.metadata['source']
            file_name = os.path.basename(file_path)
            source = f"{file_name} ({int(doc.metadata['page']) + 1}페이지)"
        sources.add(source)
    return "\n".join(sources)

def parse(ai_message: AIMessage) -> str:
    """Parse the AI message and add source."""
    return f"{ai_message.content}\n\n[출처]\n{get_metadata_sources(retrieved_docs)}"

agent_chain = (
    {"context": get_retrieved_docs, "question": RunnablePassthrough()}
    | prompt
    | eeve
    | parse
)

# agent_chain.invoke("퍼플렉시티가 투자받은 금액?") # saved_news_search / '퍼플렉시티는 투자받은 금액이 약 6300만 달러(약 860억 원)이며, 회사 가치는 10억 달러(약 1조 3760억 원) 이상으로 평가받았습니다.'
# agent_chain.invoke("생성형 AI 도입에 따른 규제 연구 책임자는?") # pdf_search / '해당 문서에 따르면, 생성형 AI 도입과 관련된 규제 연구를 담당하는 연구 책임자는 김주희 교수님입니다.'
agent_chain.invoke("라마3 성능은 어떻게 돼? 그리고 생성형 AI 도입에 따른 규제 연구 책임자는 누구야?")

get_tools / [{'tool_name': 'saved_news_search', 'tool_description': '엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다'}, {'tool_name': 'pdf_search', 'tool_description': '생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다. 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보는 이 도구를 사용해야 한다'}]
get_documents_from_actions / tool_name: saved_news_search / action_input: 라마3 성능
get_documents_from_actions / tool_name: pdf_search / action_input: 생성형 AI 신기술 도입에 따른 선거 규제 연구 책임자


'라마3의 성능은 객관식 문제(MMLU)와 코딩(HumanEval)에서 강점을 보이며, 특히 인간 선호도 측면에서 경쟁 모델을 앞서고 있습니다. 그러나 수학 단어 문제(MATH) 해결이나 대학원생 수준의 객관식 문제(GPQA)에서는 제미나이 프로 1.5에 비해 뒤처집니다. 라마3는 공개 후 몇 시간 만에 LLM 리더보드에서 1위를 차지하며 역대 가장 빠른 기록을 세웠습니다.\n\n라마3의 인간 평가 결과, 허깅페이스는 공개 후 몇 시간 만에 LLM 리더보드에서 1위에 올랐다고 언급했습니다. 또한 이전 라마 1과 2를 기반으로 한 모델이 3만 개 이상 출시되었으며, 라마 2 모델은 17억 번 다운로드되었다고 합니다. 그러나 라마 3는 완전한 오픈 소스가 아니며, 연구 및 상업적 용도로 사용 가능하지만 개발자가 다른 생성 모델을 훈련하는 것을 금지하고 있습니다.\n\n라마3의 안전성과 책임 있는 사용을 보장하기 위해 다양한 안전장치를 마련했습니다. 부적절한 답변 가능성을 최소화하기 위해 전문가와 자동화된 도구를 활용한 레드팀 테스트를 실시했습니다. 라마3는 라마 2보다 두 배 큰 컨텍스트 길이(8000 토큰)를 지원하며, 훈련 규모를 확대하고 지시 미세조정 과정을 거쳤습니다.\n\n생성형 AI 도입에 따른 규제 연구 책임자는 국립부경대학교 김주희 교수입니다. 공동연구자로는 차재권, 김현정, 조성복 교수가 있으며, 연구보조원은 박서현과 권수민입니다. 이 연구는 2023년도 중앙선거관리위원회 정책연구용역 과제로서, 생성형 AI 기술이 선거에 미치는 영향과 잠재적 문제점을 분석하고 규제 방안을 제안하고 있습니다.\n\n[출처]\nhttps://www.aitimes.com/news/articleView.html?idxno=158943\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (1페이지)\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (9페이지)\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (2페이지)